In [3]:
from aocd.models import Puzzle
import itertools
from collections import defaultdict
import numpy as np
from intcode import Intcode

In [47]:
puzzle = Puzzle(year = 2019, day = 11)
data = puzzle.input_data

inputs = [int(x) for x in data.split(",")]

inputs.extend([0]*10000000)

In [1]:
inputs = [1102,34915192,34915192,7,4,7,99,0]
inputs.extend([0]*10000)
class Intcode():
    def __init__(self, inputs):
        self.data = inputs
        self.original_data = inputs
        self.current_position = 0
        self.opcode = 0
        self.par1 = 0
        self.par2 = 0
        self.par3 = 0
        self.relative_base = 0
        self.outputs = []
    def parse_opcode(self, x):
        try: 
            self.par1 = int(str(x)[-3])
        except: 
            self.par1 = 0
        try: 
            self.par2 = int(str(x)[-4])
        except: 
            self.par2 = 0
        try: 
            self.par3 = int(str(x)[-5])
        except: 
            self.par3 = 0
        self.opcode = x % 100
    def translate_pars(self):
        if self.par1 == 0:
            self.par1 = self.data[self.current_position +1]
        else:
            if self.par1 == 1:
                self.par1 = self.current_position +1
            else:
                par = self.relative_base + self.data[self.current_position +1]
                self.par1 = par
        if self.par2 == 0:
            self.par2 = self.data[self.current_position + 2]
        else:
            if self.par2 == 1: 
                self.par2 = self.current_position +2
            else: 
                par = self.relative_base + self.data[self.current_position +2]
                self.par2 = par
        if self.par3 == 0:
            self.par3 = self.data[self.current_position+3]
        else: 
            if self.par3 == 1:
                self.par3 = self.current_position + 3
            par = self.relative_base + self.data[self.current_position +3]
            self.par3 = par
    def do_instructions(self):
        # add
        if self.opcode == 1:
            self.data[self.par3] = self.data[self.par1] + self.data[self.par2]
            self.current_position += 4
        # multiply
        if self.opcode == 2:
            self.data[self.par3] = self.data[self.par1] * self.data[self.par2]
            self.current_position += 4
        # input instruction
        if self.opcode == 3:
            input_a = self.get_input()
            self.data[self.par1] = input_a
            self.current_position += 2
        # output instruction
        if self.opcode == 4:
            print(self.data[self.par1])
            self.outputs.append(self.data[self.par1])
            self.current_position += 2
        # finish program
        if self.opcode == 99:
            self.current_position = -1
            return self.data
        # jump-if-true (non-zero)
        if self.opcode == 5:
            if self.data[self.par1] != 0:
                self.current_position = self.data[self.par2]
            else: 
                self.current_position += 3
        # jump-if-false (zero)
        if self.opcode == 6:
            if self.data[self.par1] == 0:
                self.current_position = self.data[self.par2]
            else: 
                self.current_position += 3
        # less than
        if self.opcode == 7:
            if self.data[self.par1] < self.data[self.par2]:
                self.data[self.par3] = 1
            else: 
                self.data[self.par3] = 0
            self.current_position += 4
        # equals
        if self.opcode == 8:
            if self.data[self.par1] == self.data[self.par2]:
                self.data[self.par3] = 1
            else:
                self.data[self.par3] = 0
            self.current_position += 4
        # adjusts relative base
        if self.opcode == 9:
            self.relative_base += self.data[self.par1]
            self.current_position += 2
    def get_input(self):
        return 0        
    def run(self):
        # fetch -> decode -> get_args -> execute
        while self.current_position >= 0:
            self.parse_opcode(self.data[self.current_position])
            if self.opcode != 99:
                self.translate_pars()
            self.do_instructions()
                
    
test = Intcode(inputs)
test.run()


1219070632396864


In [68]:
class RobotIntCode(Intcode):
    def __init__(self, inputs):
        Intcode.__init__(self, inputs)
        self.robot_position = (0,0)
        self.robot_direction = 0
        self.grid = {}
        self.painted = 0
        self.halt = False
        self.counter = 0
        self.instructions = 0

    def get_input(self):
        try: 
            input_a = self.grid[self.robot_position]
            print("Panel is black")
        except KeyError:
            input_a = 0
        return input_a
    
    def run_robot(self):
        color, direction = self.outputs
        try: 
            old_color = self.grid[self.robot_position]
        except KeyError: 
            self.painted += 1
        self.grid[self.robot_position] = color
        #print(self.robot_position)
        if direction == 0:
            #direction 
            self.robot_direction = (self.robot_direction - 90) % 360
        else: 
            self.robot_direction = (self.robot_direction + 90) % 360
        if self.robot_direction == 0:
            self.robot_position = (self.robot_position[0]+1, self.robot_position[1])
        if self.robot_direction == 90:
            self.robot_position = (self.robot_position[0], self.robot_position[1]+1)
        if self.robot_direction == 180:
            self.robot_position = (self.robot_position[0]-1, self.robot_position[1])
        if self.robot_direction == 270:
            self.robot_position = (self.robot_position[0], self.robot_position[1]-1)
        self.halt = False
    
    def run(self, length_outputs):
        # fetch -> decode -> get_args -> execute
        while self.current_position >= 0:
            self.parse_opcode(self.data[self.current_position])
            if self.opcode != 99:
                self.translate_pars()
            self.do_instructions()
            self.instructions += 1
            if len(self.outputs) == length_outputs: 
                self.counter += 1
                self.halt = True
                self.run_robot()
                self.outputs = []

puzzle = Puzzle(year = 2019, day = 11)
data = puzzle.input_data

inputs = [int(x) for x in data.split(",")]

inputs.extend([0]*10000)
test = RobotIntCode(inputs)
test.run(2)

#6523
print(test.painted)
print(test.instructions)
print(len(test.grid))
print(test.counter)
        
        

1
0
1
1
1
0
1
1
1
1
1
0
1
1
1
1
1
0
1
0
1
0
Panel is black
0
0
Panel is black
0
0
Panel is black
0
0
Panel is black
0
0
Panel is black
1
1
1
1
1
1
Panel is black
1
1
Panel is black
0
0
Panel is black
1
1
Panel is black
0
1
Panel is black
0
1
Panel is black
1
0
Panel is black
0
1
Panel is black
0
0
1
1
1
1
1
1
Panel is black
1
0
Panel is black
0
0
1
0
Panel is black
0
1
1
1
1
0
1
0
1
1
1
1
1
1
Panel is black
0
0
1
0
1
1
1
0
1
1
1
1
1
1
Panel is black
0
0
1
1
Panel is black
0
0
1
0
1
1
1
1
1
1
Panel is black
0
0
Panel is black
0
0
Panel is black
0
0
Panel is black
0
1
Panel is black
1
1
Panel is black
1
0
Panel is black
0
0
Panel is black
0
0
Panel is black
0
0
Panel is black
0
0
Panel is black
1
0
Panel is black
1
0
Panel is black
1
0
Panel is black
1
0
Panel is black
0
0
Panel is black
0
0
Panel is black
0
1
1
0
Panel is black
0
1
1
0
1
1
1
1
1
1
Panel is black
0
0
1
0
1
0
Panel is black
0
1
1
1
1
0
1
1
1
1
1
1
Panel is black
0
0
Panel is black
0
1
Panel is black
1
1
Panel is black
0
0

1
Panel is black
0
1
Panel is black
0
1
Panel is black
1
0
Panel is black
0
1
Panel is black
1
1
Panel is black
1
1
Panel is black
0
0
Panel is black
1
1
Panel is black
1
0
Panel is black
1
0
Panel is black
0
1
Panel is black
0
1
Panel is black
1
1
Panel is black
0
1
Panel is black
1
1
Panel is black
1
1
Panel is black
0
1
Panel is black
1
1
Panel is black
0
0
Panel is black
0
0
Panel is black
0
1
Panel is black
1
0
Panel is black
1
1
Panel is black
0
1
Panel is black
1
1
Panel is black
0
0
Panel is black
1
0
Panel is black
1
1
Panel is black
0
1
Panel is black
0
1
Panel is black
0
1
Panel is black
0
0
Panel is black
0
0
Panel is black
1
0
Panel is black
1
1
Panel is black
1
0
Panel is black
1
1
Panel is black
1
1
Panel is black
1
0
Panel is black
0
1
Panel is black
1
0
Panel is black
0
1
Panel is black
0
1
Panel is black
0
0
Panel is black
0
0
Panel is black
0
0
Panel is black
1
1
1
1
1
1
Panel is black
1
0
1
1
Panel is black
1
0
Panel is black
1
0
1
0
Panel is black
0
1
1
0
Panel is 

1
1
1
Panel is black
1
1
Panel is black
1
1
Panel is black
0
1
Panel is black
0
0
1
0
1
1
1
0
1
0
1
1
1
1
1
1
Panel is black
0
0
1
0
1
0
Panel is black
0
0
Panel is black
1
1
Panel is black
0
0
Panel is black
0
0
Panel is black
0
0
Panel is black
0
0
Panel is black
1
1
Panel is black
0
1
Panel is black
1
1
Panel is black
1
1
Panel is black
0
1
Panel is black
1
1
Panel is black
0
1
Panel is black
0
1
Panel is black
1
0
Panel is black
1
0
Panel is black
1
1
1
0
1
1
1
1
1
0
1
1
1
0
1
0
1
1
1
1
1
1
Panel is black
0
0
1
1
1
1
Panel is black
0
0
1
1
Panel is black
0
0
Panel is black
0
0
Panel is black
0
0
Panel is black
0
0
Panel is black
1
1
Panel is black
0
1
Panel is black
1
1
Panel is black
1
1
Panel is black
0
1
Panel is black
1
1
Panel is black
0
1
Panel is black
0
1
Panel is black
1
0
Panel is black
1
0
Panel is black
1
1
Panel is black
0
1
Panel is black
0
0
Panel is black
1
1
Panel is black
0
1
1
1
Panel is black
1
0
Panel is black
0
1
Panel is black
0
0
Panel is black
1
1
1
1
Panel

1
Panel is black
1
1
Panel is black
0
1
Panel is black
1
1
Panel is black
0
0
Panel is black
1
1
Panel is black
0
0
1
0
Panel is black
0
0
Panel is black
0
1
Panel is black
0
0
Panel is black
0
0
Panel is black
1
0
Panel is black
1
1
Panel is black
1
0
Panel is black
0
0
Panel is black
1
0
Panel is black
0
0
Panel is black
0
1
Panel is black
0
1
Panel is black
0
1
Panel is black
1
0
Panel is black
1
1
Panel is black
1
1
Panel is black
0
0
1
0
1
1
1
0
1
0
1
0
Panel is black
0
1
Panel is black
0
0
Panel is black
1
1
Panel is black
0
0
Panel is black
1
0
Panel is black
1
1
1
1
1
1
Panel is black
0
0
Panel is black
1
1
Panel is black
1
0
Panel is black
0
1
Panel is black
1
1
Panel is black
1
0
1
1
Panel is black
1
1
Panel is black
0
0
Panel is black
1
1
Panel is black
0
1
Panel is black
1
1
Panel is black
1
1
Panel is black
0
1
Panel is black
1
1
Panel is black
0
0
Panel is black
1
1
Panel is black
0
0
1
0
Panel is black
0
0
Panel is black
0
1
Panel is black
0
0
Panel is black
0
0
Panel is

Panel is black
1
0
Panel is black
1
0
Panel is black
0
0
Panel is black
1
1
Panel is black
1
1
Panel is black
0
0
Panel is black
0
1
Panel is black
0
1
Panel is black
0
0
Panel is black
0
1
Panel is black
0
0
Panel is black
0
0
Panel is black
0
1
Panel is black
1
1
Panel is black
0
0
Panel is black
0
1
Panel is black
0
1
Panel is black
1
0
Panel is black
1
0
Panel is black
0
1
Panel is black
1
0
Panel is black
1
0
Panel is black
0
1
Panel is black
1
1
Panel is black
0
1
Panel is black
0
1
Panel is black
1
0
Panel is black
1
1
Panel is black
1
1
Panel is black
0
1
Panel is black
0
0
Panel is black
0
0
Panel is black
0
1
Panel is black
1
1
Panel is black
0
1
Panel is black
1
0
Panel is black
1
1
Panel is black
1
1
Panel is black
1
1
Panel is black
0
1
Panel is black
0
1
Panel is black
0
1
Panel is black
0
1
Panel is black
1
1
Panel is black
1
0
1
1
1
1
Panel is black
0
0
Panel is black
0
0
1
0
Panel is black
0
1
1
0
1
0
Panel is black
0
0
Panel is black
1
1
Panel is black
1
1
Panel is bl

...

1
Panel is black
1
1
Panel is black
1
0
Panel is black
0
1
Panel is black
0
0
Panel is black
0
1
Panel is black
0
0
Panel is black
0
0
Panel is black
0
1
Panel is black
1
0
Panel is black
0
1
1
1
1
1
Panel is black
0
1
Panel is black
1
0
Panel is black
0
1
Panel is black
1
0
Panel is black
1
0
Panel is black
0
1
Panel is black
1
1
Panel is black
0
1
Panel is black
0
0
Panel is black
1
1
Panel is black
0
1
Panel is black
0
0
1
0
1
1
1
1
1
0
1
1
1
0
1
0
1
1
1
0
1
0
1
1
1
1
1
1
Panel is black
0
0
1
1
1
1
Panel is black
0
0
1
1
Panel is black
0
0
1
1
1
1
Panel is black
0
0
Panel is black
0
0
Panel is black
0
1
Panel is black
1
1
Panel is black
0
0
Panel is black
1
0
Panel is black
1
1
Panel is black
1
0
Panel is black
1
1
Panel is black
0
0
Panel is black
0
1
Panel is black
1
0
Panel is black
0
1
Panel is black
1
1
Panel is black
0
1
Panel is black
0
0
Panel is black
1
1
Panel is black
1
1
Panel is black
1
1
Panel is black
1
0
Panel is black
1
0
Panel is black
0
0
Panel is black
0
1
1
1
1


Panel is black
0
1
1
1
1
0
1
1
1
1
1
0
1
1
1
0
1
0
1
1
1
0
1
1
1
1
1
1
Panel is black
0
0
1
1
Panel is black
0
0
1
1
1
1
Panel is black
0
0
1
1
Panel is black
0
0
Panel is black
0
0
Panel is black
0
0
Panel is black
0
1
Panel is black
0
0
Panel is black
0
1
1
1
Panel is black
0
0
Panel is black
0
1
Panel is black
1
1
Panel is black
1
0
Panel is black
0
1
Panel is black
1
0
Panel is black
0
1
Panel is black
1
0
Panel is black
1
0
Panel is black
1
1
Panel is black
1
0
Panel is black
0
1
Panel is black
1
1
Panel is black
1
1
Panel is black
0
1
Panel is black
1
1
Panel is black
0
1
Panel is black
0
0
Panel is black
1
1
Panel is black
0
0
Panel is black
0
0
Panel is black
0
1
Panel is black
0
0
Panel is black
0
0
Panel is black
0
1
Panel is black
1
1
Panel is black
0
1
Panel is black
1
0
Panel is black
0
0
Panel is black
0
1
Panel is black
0
1
Panel is black
0
1
Panel is black
1
0
Panel is black
1
0
Panel is black
0
1
Panel is black
0
0
Panel is black
1
0
Panel is black
0
0
Panel is black
1

0
1
1
Panel is black
0
0
1
0
1
1
1
1
1
1
Panel is black
0
0
Panel is black
0
0
Panel is black
0
1
Panel is black
0
1
Panel is black
0
0
Panel is black
0
1
Panel is black
1
1
Panel is black
1
1
Panel is black
1
1
Panel is black
1
1
Panel is black
0
1
Panel is black
0
1
Panel is black
0
1
Panel is black
0
1
Panel is black
1
0
Panel is black
1
0
Panel is black
1
1
Panel is black
0
0
Panel is black
0
0
Panel is black
0
0
Panel is black
0
1
Panel is black
1
0
Panel is black
0
1
Panel is black
1
0
Panel is black
0
0
Panel is black
1
1
Panel is black
0
0
Panel is black
0
1
Panel is black
0
1
Panel is black
0
1
Panel is black
1
0
Panel is black
0
0
Panel is black
1
0
Panel is black
1
1
Panel is black
0
1
Panel is black
1
1
Panel is black
0
1
Panel is black
0
1
Panel is black
1
0
Panel is black
0
1
1
1
1
0
1
1
1
1
1
0
1
1
Panel is black
0
1
Panel is black
0
1
Panel is black
0
0
Panel is black
0
1
Panel is black
0
0
1
1
1
1
1
1
Panel is black
1
1
Panel is black
0
0
Panel is black
0
1
Panel is bl

1
1
Panel is black
1
0
Panel is black
0
0
Panel is black
1
0
Panel is black
0
1
Panel is black
1
1
Panel is black
1
0
Panel is black
0
1
Panel is black
1
0
Panel is black
0
0
Panel is black
1
1
Panel is black
1
1
Panel is black
0
1
Panel is black
1
1
Panel is black
0
1
Panel is black
0
0
Panel is black
0
0
Panel is black
1
0
Panel is black
1
1
Panel is black
1
0
Panel is black
0
0
Panel is black
0
0
Panel is black
0
1
Panel is black
1
1
Panel is black
1
0
Panel is black
0
1
Panel is black
1
0
Panel is black
0
0
1
1
1
1
1
0
1
0
1
0
Panel is black
0
0
Panel is black
0
0
Panel is black
0
1
1
1
1
0
1
1
1
1
Panel is black
0
0
Panel is black
0
0
1
1
Panel is black
0
1
Panel is black
0
1
Panel is black
1
0
Panel is black
0
0
Panel is black
1
1
Panel is black
0
0
Panel is black
1
1
Panel is black
1
0
Panel is black
0
1
Panel is black
0
0
Panel is black
0
1
Panel is black
0
1
Panel is black
0
0
Panel is black
0
1
Panel is black
1
1
Panel is black
1
0
Panel is black
0
0
Panel is black
0
0
Panel 

0
Panel is black
0
0
Panel is black
1
0
Panel is black
0
1
Panel is black
1
0
Panel is black
1
1
Panel is black
1
1
Panel is black
0
0
Panel is black
1
1
Panel is black
0
0
Panel is black
0
1
Panel is black
1
0
Panel is black
0
0
Panel is black
0
1
Panel is black
1
1
Panel is black
1
1
Panel is black
1
1
Panel is black
1
0
Panel is black
1
1
Panel is black
1
0
Panel is black
0
1
Panel is black
1
1
Panel is black
0
1
Panel is black
0
1
Panel is black
1
1
Panel is black
0
0
Panel is black
0
0
Panel is black
1
1
Panel is black
0
0
Panel is black
0
0
Panel is black
1
0
Panel is black
0
0
Panel is black
1
0
Panel is black
1
0
Panel is black
0
0
Panel is black
1
0
Panel is black
0
1
Panel is black
0
0
Panel is black
0
1
Panel is black
0
1
Panel is black
1
1
Panel is black
1
0
Panel is black
1
1
Panel is black
0
0
Panel is black
0
1
Panel is black
1
1
Panel is black
1
0
Panel is black
1
0
Panel is black
0
1
Panel is black
1
0
Panel is black
1
1
Panel is black
0
0
1
1
1
0
1
0
1
1
Panel is blac

In [43]:
class RobotIntCode(Intcode):
    def __init__(self, inputs):
        Intcode.__init__(self, inputs)
        self.robot_position = (0,0)
        self.robot_direction = 0
        self.grid = {(0,0):1}
        self.painted = 0
        self.halt = False
        self.counter = 0
        self.instructions = 0

    def get_input(self):
        try: 
            input_a = self.grid[self.robot_position]
            print("Panel is black")
        except KeyError:
            input_a = 0
        return input_a
    
    def run_robot(self):
        color, direction = self.outputs
        try: 
            old_color = self.grid[self.robot_position]
        except KeyError: 
            self.painted += 1
        self.grid[self.robot_position] = color
        #print(self.robot_position)
        if direction == 0:
            #direction 
            self.robot_direction = (self.robot_direction - 90) % 360
        else: 
            self.robot_direction = (self.robot_direction + 90) % 360
        if self.robot_direction == 0:
            self.robot_position = (self.robot_position[0]+1, self.robot_position[1])
        if self.robot_direction == 90:
            self.robot_position = (self.robot_position[0], self.robot_position[1]+1)
        if self.robot_direction == 180:
            self.robot_position = (self.robot_position[0]-1, self.robot_position[1])
        if self.robot_direction == 270:
            self.robot_position = (self.robot_position[0], self.robot_position[1]-1)
        self.halt = False
    
    def run(self, length_outputs):
        # fetch -> decode -> get_args -> execute
        while self.current_position >= 0:
            self.parse_opcode(self.data[self.current_position])
            if self.opcode != 99:
                self.translate_pars()
            self.do_instructions()
            self.instructions += 1
            if len(self.outputs) == length_outputs: 
                self.counter += 1
                self.halt = True
                self.run_robot()
                self.outputs = []

puzzle = Puzzle(year = 2019, day = 11)
data = puzzle.input_data

inputs = [int(x) for x in data.split(",")]

inputs.extend([0]*10000)
test = RobotIntCode(inputs)
test.run(2)

#6523
print(test.painted)
print(test.instructions)
print(len(test.grid))
print(test.counter)


Panel is black
0
1
1
1
1
0
0
0
0
1
0
1
0
0
1
0
1
1
0
1
0
0
0
0
1
1
1
1
0
0
0
0
1
1
1
1
1
0
0
0
0
1
0
1
1
0
0
0
1
1
1
1
0
0
1
0
0
1
0
1
0
0
1
0
1
1
1
1
0
0
0
0
1
1
1
1
0
0
0
0
0
1
1
1
1
0
0
0
0
1
0
1
1
0
0
0
1
1
0
1
0
0
1
0
1
1
1
1
0
0
0
0
1
1
0
1
1
0
0
0
0
1
1
1
1
0
0
0
0
1
0
1
0
0
0
0
0
1
0
1
0
0
1
0
1
1
1
1
0
0
0
0
1
1
0
1
1
0
0
0
0
1
0
1
0
0
0
1
0
1
0
0
0
1
0
1
0
0
1
0
0
1
1
1
0
0
0
0
1
1
1
1
1
0
0
0
0
1
0
1
0
0
0
0
0
1
0
1
0
0
1
0
1
1
0
1
0
0
0
0
1
1
0
1
1
0
1
0
0
1
1
1
1
0
0
0
0
1
0
1
0
0
0
0
1
1
0
1
1
0
1
0
1
1
0
1
0
0
0
0
0
1
0
1
1
0
1
0
0
1
1
1
1
0
0
0
0
1
1
1
1
0
0
0
1
1
1
1
0
0
1
0
1
1
0
1
0
0
0
0
0
1
0
1
1
0
0
0
1
1
0
1
0
0
0
0
0
1
1
1
1
0
0
0
0
1
0
1
0
0
1
0
1
1
0
0
0
0
1
1
0
0
1
0
0
1
0
1
1
0
0
0
1
1
0
1
0
0
1
0
1
1
0
1
1
0
1
0
0
1
0
1
1
0
0
0
0
1
1
1
1
0
0
0
0
1
0
1
0
0
1
0
1
1
0
1
0
0
1
0
1
1
0
1
1
0
1
0
0
1
0
1
1
0
0
0
0
1
1
1
1
0
0
0
0
1
1
1
0
0
1
0
0
1
0
1
0
0
1
0
1
1
0
1
1
0
1
0
0
1
1
1
0
0
0
0
0
1
1
1
1
0
1
0
0
1
0
1
1
0
1
0
0
1
0
1
0
0
1
0
1
1
0
1
0
0
0
0
0
1
0
1
0

In [44]:
from PIL import Image

pic = test.grid.copy()

for key in list(pic.keys()):
    new_key = (key[0]+5, key[1])
    pic[new_key] = pic.pop(key)
    
print(pic)
    
#im_map[1] =  (255,255,255)
#im_map[0] =  (0, 0, 0)

im = Image.new("1", (50, 50))

for key in list(pic.keys()):
    im.putpixel((key[0], key[1]), pic[key])
    im.rotate(90)
    
print(im.rotate(90).show())



{(5, 0): 0, (5, 1): 1, (4, 1): 1, (4, 2): 0, (5, 2): 0, (5, 3): 0, (4, 3): 0, (4, 4): 1, (5, 4): 1, (5, 5): 0, (4, 5): 0, (4, 6): 0, (5, 6): 1, (5, 7): 1, (4, 7): 0, (4, 8): 0, (5, 8): 1, (5, 9): 1, (4, 9): 1, (4, 10): 0, (5, 10): 0, (5, 11): 0, (4, 11): 1, (4, 12): 0, (5, 12): 1, (5, 13): 1, (4, 13): 0, (4, 14): 1, (5, 14): 0, (5, 15): 0, (4, 15): 0, (4, 16): 1, (5, 16): 1, (5, 17): 1, (4, 17): 0, (4, 18): 0, (5, 18): 1, (5, 19): 1, (4, 19): 0, (4, 20): 0, (5, 20): 0, (5, 21): 1, (4, 21): 1, (4, 22): 0, (5, 22): 0, (5, 23): 0, (4, 23): 1, (4, 24): 0, (5, 24): 1, (5, 25): 0, (4, 25): 0, (4, 26): 1, (5, 26): 1, (5, 27): 1, (4, 27): 0, (4, 28): 0, (5, 28): 1, (5, 29): 0, (4, 29): 1, (4, 30): 0, (5, 30): 0, (5, 31): 1, (4, 31): 1, (4, 32): 0, (5, 32): 0, (5, 33): 0, (4, 33): 0, (4, 34): 0, (5, 34): 0, (5, 35): 0, (4, 35): 0, (4, 36): 1, (5, 36): 1, (5, 37): 1, (4, 37): 0, (4, 38): 0, (5, 38): 1, (5, 39): 0, (4, 39): 1, (4, 40): 0, (5, 40): 0, (5, 41): 0, (4, 41): 0, (4, 42): 0, (3, 42): 0

## Tag 18

In [102]:
test_input = """#########
#b.A.@.a#
#########"""


[list(x) for x in test_input.split("\n")]



{(5, 0): 0, (5, 1): 1, (4, 1): 1, (4, 2): 0, (5, 2): 0, (5, 3): 0, (4, 3): 0, (4, 4): 1, (5, 4): 1, (5, 5): 0, (4, 5): 0, (4, 6): 0, (5, 6): 1, (5, 7): 1, (4, 7): 0, (4, 8): 0, (5, 8): 1, (5, 9): 1, (4, 9): 1, (4, 10): 0, (5, 10): 0, (5, 11): 0, (4, 11): 1, (4, 12): 0, (5, 12): 1, (5, 13): 1, (4, 13): 0, (4, 14): 1, (5, 14): 0, (5, 15): 0, (4, 15): 0, (4, 16): 1, (5, 16): 1, (5, 17): 1, (4, 17): 0, (4, 18): 0, (5, 18): 1, (5, 19): 1, (4, 19): 0, (4, 20): 0, (5, 20): 0, (5, 21): 1, (4, 21): 1, (4, 22): 0, (5, 22): 0, (5, 23): 0, (4, 23): 1, (4, 24): 0, (5, 24): 1, (5, 25): 0, (4, 25): 0, (4, 26): 1, (5, 26): 1, (5, 27): 1, (4, 27): 0, (4, 28): 0, (5, 28): 1, (5, 29): 0, (4, 29): 1, (4, 30): 0, (5, 30): 0, (5, 31): 1, (4, 31): 1, (4, 32): 0, (5, 32): 0, (5, 33): 0, (4, 33): 0, (4, 34): 0, (5, 34): 0, (5, 35): 0, (4, 35): 0, (4, 36): 1, (5, 36): 1, (5, 37): 1, (4, 37): 0, (4, 38): 0, (5, 38): 1, (5, 39): 0, (4, 39): 1, (4, 40): 0, (5, 40): 0, (5, 41): 0, (4, 41): 0, (4, 42): 0, (3, 42): 0

# Tag 14

In [41]:
inputs = """9 ORE => 2 A
8 ORE => 3 B
7 ORE => 5 C
3 A, 4 B => 1 AB
5 B, 7 C => 1 BC
4 C, 1 A => 1 CA
2 AB, 3 BC, 4 CA => 1 FUEL"""



inputs = """10 ORE => 10 A
1 ORE => 1 B
7 A, 1 B => 1 C
7 A, 1 C => 1 D
7 A, 1 D => 1 E
7 A, 1 E => 1 FUEL"""

data = inputs.split("\n")
all_inputs = [x.split("=>")[0].strip().split(", ") for x in data]
print(all_inputs)
all_outputs = [x.split("=>")[1].strip() for x in data]

### if the thing I'm looking for is in outputs: produce x of this thing: 

def get_ingredients(outputs, final):
    for element in outputs: 
        if final == element.split(" ")[1]:
            return []
    new_outputs = []
    inputs = []
    inputs_index = []
    for i, element in enumerate(outputs): 
        print(element)
        product = element.split(" ")[1]
        amount = element.split(" ")[0]
        for j, element in enumerate(all_outputs):
            if product == element.split(" ")[1]:
                inputs.extend(all_inputs[j])
                inputs.append(i)
                inputs.append(j)
                print(inputs)

    for i, one_input in enumerate(inputs[::3]):
        print(one_input)
        input_product = one_input.split(" ")[1]
        outputs_index = inputs[i+2]
        all_outputs_index = inputs[i+1]
        ## the count calculation does not work currently
        get_count(one_input, outputs_index, all_outputs_index)
        #count = int(one_input.split(" ")[0]) * int(
         #   outputs[all_outputs_index].split(" ")[0]) / int(
          #  all_outputs[outputs_index].split(" ")[0])
        new_outputs.append(f"{count} {input_product}")
    return new_outputs
    
    
    
get_ingredients(["1 FUEL"], "ORE")

def get_count(one_input_product, outputs_index, all_outputs_index):
    pass
    

outputs = ["1 FUEL"]
while outputs: 
    for element in outputs: 
        outputs = get_ingredients(outputs, "ORE")
        print(outputs)


[['10 ORE'], ['1 ORE'], ['7 A', '1 B'], ['7 A', '1 C'], ['7 A', '1 D'], ['7 A', '1 E']]
1 FUEL
['7 A', '1 E', 0, 5]
7 A


NameError: name 'get_count' is not defined

# Tag 17

In [ ]:
view = """..#..........
..#..........
#######...###
#.#...#...#.#
#############
..#...#...#..
..#####...^.."""

view_list = [list(x) for x in view.split("\n")]

In [37]:
day17_puzzle = Puzzle(year=2019, day=17)
input_day17 = day17_puzzle.input_data
input_day17 = [int(x) for x in input_day17.split(",")]
input_day17.extend([0]*10000)
ascii_17 = Intcode(input_day17)
ascii_17.run()
view = ""
for element in ascii_17.outputs:
    view += str(chr(element))
view_list = [list(x) for x in view.split("\n")]
#print(view_list)



sum_ap = 0
for i in range(1, len(view_list)-3):
    for j in range(1, len(view_list[i])-1):
        if view_list[i][j] == "#":
            if (view_list[i+1][j] == "#" and view_list[i-1][j] == "#" 
                and view_list[i][j-1] == "#" and view_list[i][j+1] == "#"):
                alignment_parameter = i*j
                sum_ap += alignment_parameter
                
print(sum_ap)
                

46
46
46
46
46
46
94
35
35
35
35
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
10
46
46
46
46
46
46
46
46
46
46
35
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
10
46
46
46
46
46
46
46
46
46
46
35
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
10
46
46
46
46
46
46
46
46
46
46
35
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
10
46
46
46
46
46
46
46
46
46
46
35
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
10
46
46
46
46
46
46
46
46
46
46
35
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
46
10
46
46
35
35
35
35
35
35
35
35
35
35
35
46
35
35
35
35
35
35
35
46
46
46
46
46
46
46
46
10
46
46
35
46
46
46
46
46
46
46
35
46
35
46
35
46
46
46
46
46
35
46
46
46
46
46
46
46
46
10
46
46
35
46
46
46
46
46
35
35
35
35
35
35
35
35
35
35
35
46
35
46
46
46
46
46
46
46
46
10
46
46
35
46
46
46
46
46
35
46
35
46
35
46
35
46
46
46
35
46
35
46
46
46
46
46
46
46
46
10
46
46
35
35
35
35
35
35
35
35
35
46
35
46
35
46
35
35
35
35
35
46
46
46
46
46
46
46
46
10
46
46
46
4

## Tag 24

In [24]:
initial_state = """....#
#..#.
#..##
..#..
#...."""

inputs = [list(x) for x in initial_state.split("\n")]




def get_neighbor_count(x, i, j):
    neighbors = []
    if i < len(x) -1:
        neighbors.append(x[i+1][j])
    if i > 0:
        neighbors.append(x[i-1][j])
    if j < len(x[i])-1:
        neighbors.append(x[i][j+1])
    if j > 0:
        neighbors.append(x[i][j-1])
    return len([x for x in neighbors if x == "#"])

def get_current_state(el, neighbor_count):
    if el == "#" and neighbor_count != 1:
        el = "."
    elif el == "." and neighbor_count in (1, 2):
        el = "#"
    return el

def step(inputs):
    after_list = []
    for i, lst in enumerate(inputs):
        lst_i = []
        for j, element in enumerate(lst):
            n_count = get_neighbor_count(inputs, i, j)
            state = get_current_state(element, n_count)
            lst_i.append(state)
        after_list.append(lst_i)
    return after_list

def flatten_list(lst):
    flat_list = []
    for element in lst:
        flat_list.extend(element)
    return flat_list

#print(flatten_list(inputs))

def calculate_biodiversity(lst):
    flat_list = flatten_list(lst)
    result = 0
    for i, element in enumerate(flat_list): 
        if element == "#":
            result += 2**i
    return result

def run(inputs):
    i = 0
    after_list = []
    steps_lst = [inputs]
    while after_list not in steps_lst:
        after_list = step(steps_lst[i])
        if after_list not in steps_lst:
            steps_lst.append(after_list)
            print(after_list)
            after_list = []
            i += 1
        else: 
            bd = calculate_biodiversity(after_list)
            print(f"Done:{after_list}")
            return bd

print(run(inputs))
        


[['#', '.', '.', '#', '.'], ['#', '#', '#', '#', '.'], ['#', '#', '#', '.', '#'], ['#', '#', '.', '#', '#'], ['.', '#', '#', '.', '.']]
[['#', '#', '#', '#', '#'], ['.', '.', '.', '.', '#'], ['.', '.', '.', '.', '#'], ['.', '.', '.', '#', '.'], ['#', '.', '#', '#', '#']]
[['#', '.', '.', '.', '.'], ['#', '#', '#', '#', '.'], ['.', '.', '.', '#', '#'], ['#', '.', '#', '#', '.'], ['.', '#', '#', '.', '#']]
[['#', '#', '#', '#', '.'], ['.', '.', '.', '.', '#'], ['#', '#', '.', '.', '#'], ['.', '.', '.', '.', '.'], ['#', '#', '.', '.', '.']]
[['#', '.', '.', '#', '#'], ['#', '#', '#', '#', '#'], ['#', '#', '#', '#', '#'], ['#', '#', '.', '.', '#'], ['#', '#', '#', '.', '.']]
[['#', '#', '#', '.', '.'], ['.', '.', '.', '.', '.'], ['.', '.', '.', '.', '.'], ['.', '.', '.', '#', '#'], ['.', '.', '#', '#', '#']]
[['#', '.', '#', '#', '.'], ['#', '#', '#', '.', '.'], ['.', '.', '.', '#', '#'], ['.', '.', '#', '.', '.'], ['.', '#', '#', '.', '.']]
[['#', '.', '.', '#', '#'], ['.', '.', '.', '.',

In [25]:
puzzle = Puzzle(year = 2019, day = 24)
data = puzzle.input_data

inputs_right = [list(x) for x in data.split("\n")]

print(inputs)
print(run(inputs_right))

[['.', '.', '.', '.', '#'], ['#', '.', '.', '#', '.'], ['#', '.', '.', '#', '#'], ['.', '.', '#', '.', '.'], ['#', '.', '.', '.', '.']]
[['#', '.', '.', '#', '.'], ['#', '#', '#', '.', '.'], ['.', '#', '#', '.', '#'], ['#', '.', '.', '.', '.'], ['#', '#', '.', '.', '.']]
[['#', '#', '#', '.', '#'], ['.', '.', '.', '#', '#'], ['.', '.', '.', '#', '.'], ['#', '.', '#', '.', '#'], ['.', '#', '#', '.', '.']]
[['#', '.', '#', '.', '#'], ['#', '#', '#', '.', '.'], ['#', '.', '#', '#', '.'], ['.', '.', '#', '.', '.'], ['#', '#', '.', '#', '#']]
[['#', '.', '#', '#', '.'], ['.', '.', '.', '#', '#'], ['#', '.', '.', '#', '#'], ['#', '#', '#', '.', '#'], ['#', '#', '.', '#', '#']]
[['.', '#', '#', '.', '#'], ['#', '.', '#', '.', '.'], ['#', '#', '#', '.', '.'], ['.', '.', '#', '.', '.'], ['.', '.', '.', '#', '.']]
[['#', '#', '.', '#', '.'], ['#', '.', '.', '#', '#'], ['.', '.', '.', '#', '.'], ['#', '#', '#', '#', '.'], ['.', '.', '#', '.', '#']]
[['.', '#', '#', '#', '#'], ['#', '#', '#', '.',